In [1]:
import GLPE
#https://github.com/ekehoe32/orthrus
import sys
sys.path.append('/home/katrina/a/mankovic/ZOETIS/Fall2021/Orthrus/orthrus')
import orthrus
from orthrus import core
from orthrus.core import dataset
import numpy as np
# from NetworkDataAnalysis import graph_tools_construction as gt
from matplotlib import pyplot as plt
import pandas
# from orthrus.core.pipeline import *
from sklearn.preprocessing import FunctionTransformer
from orthrus.preprocessing.imputation import HalfMinimum
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from orthrus.core.helper import load_object

In [2]:
#load incidence data
pathway_edges = pandas.read_csv('/data3/darpa/omics_databases/ensembl2pathway/reactome_human_pathway_edges.csv').dropna()

incidence_data = np.array(pathway_edges)[:,[2,4,5,7]]

node_ids = np.unique(incidence_data[:,:2])
translate_dict = { node_ids[i] :i  for i in range(len(node_ids))}
incidence_data[:,:2] = np.vectorize(translate_dict.get)(incidence_data[:,:2])

incidence_data

array([[4013, 1687, 'directed', 'R-HSA-1059683'],
       [4013, 1687, 'undirected', 'R-HSA-1059683'],
       [4013, 1594, 'directed', 'R-HSA-1059683'],
       ...,
       [7555, 1727, 'undirected', 'R-HSA-997272'],
       [7555, 1728, 'undirected', 'R-HSA-997272'],
       [7555, 1118, 'undirected', 'R-HSA-997272']], dtype=object)

In [3]:
my_clpe = GLPE.CLPE('degree', 'precomputed',incidence_data, heat_kernel_param = 2)
my_clpe

CLPE(centrality_measure='degree', heat_kernel_param=2.0,
     incidence_matrix=array([[4013, 1687, 'directed', 'R-HSA-1059683'],
       [4013, 1687, 'undirected', 'R-HSA-1059683'],
       [4013, 1594, 'directed', 'R-HSA-1059683'],
       ...,
       [7555, 1727, 'undirected', 'R-HSA-997272'],
       [7555, 1728, 'undirected', 'R-HSA-997272'],
       [7555, 1118, 'undirected', 'R-HSA-997272']], dtype=object),
     network_type='precomputed')

In [4]:
my_clpe.fit()

[404 1594 1595 1596 1686 1687 2652 3141 3143 3388 4013]


IndexError: arrays used as indices must be of integer (or boolean) type

array([1, 2])

In [ ]:
#load data
Z40_dataset = dataset.load_dataset('/data4/mankovic/De-Identified_CZ/DeId_TPM_C1_Z40_Z34.ds')
Z40_dataset.metadata = Z40_dataset.metadata.query("Project == 'Z40' & Treatment == 'High' & Timepoint == 0.0")
sidx = list(Z40_dataset.metadata.index)
Z40_dataset.data = Z40_dataset.data.loc[sidx]

#transform data (according to Kartikay)
transform = make_pipeline(HalfMinimum(missing_values=0), FunctionTransformer(np.log2))
Z40_dataset.normalize(transform, norm_name='HalfMinimum + Log2')
Z40_data = Z40_dataset.data

#which genes are in which pathways
pathway_data = pandas.read_csv('/data4/mankovic/De-Identified_CZ/deidentified_fcpw.csv')

#restrict pathway data to to genes that are actually there
pathway_data = pathway_data[['RandID']+list(Z40_data.columns)]


In [ ]:
better_pathway_data=pandas.DataFrame(columns = ['feature_id', 'pathway_id'])
gene_names = pathway_data.columns
for row in np.array(pathway_data):
    idx = np.where(row == True)
    # print(row)
    for g in gene_names[idx]:
        better_pathway_data = better_pathway_data.append({'feature_id': int(g), 'pathway_id':row[0]}, ignore_index = True)


In [ ]:
better_pathway_data = np.array(better_pathway_data)

node_ids = np.unique(better_pathway_data[:,0])

translate_dict = { node_ids[i] :i  for i in range(len(node_ids))}
better_pathway_data[:,0] = np.vectorize(translate_dict.get)(better_pathway_data[:,0])

In [5]:
np.unique(better_pathway_data[:,0])

NameError: name 'better_pathway_data' is not defined

In [6]:
my_clpe = GLPE.CLPE('degree', 
                    'correlation',
                    better_pathway_data, 
                    heat_kernel_param = 2)
my_clpe

NameError: name 'better_pathway_data' is not defined

In [94]:
small_dataset = np.array(Z40_data[node_ids.astype(str)])
small_dataset.shape

(8, 8800)

In [95]:
my_clpe.fit(small_dataset)

IndexError: arrays used as indices must be of integer (or boolean) type

,2,3,7,21,25,32,37,41,43,56,...,35545,35546,35549,35551,35554,35555,35564,35577,35586,35588
SeqID,,,,,,,,,,,,,,,,,,,,,
59,4.771260,3.306837,-0.430099,-5.877244,0.949691,3.793892,1.961236,5.145107,-11.122444,-0.137262,...,4.957056,-11.122444,1.600476,3.328092,4.375704,4.320733,3.070634,0.367303,-0.894545,3.125156
142,4.926443,3.847411,1.556034,-11.122444,1.821752,5.025138,-0.429007,4.663432,-11.122444,0.184182,...,4.977735,-11.122444,-0.608851,3.449023,4.531042,4.642691,2.682643,-0.575481,-1.971421,3.190132
32,5.185960,1.998647,-0.844895,-11.122444,-1.079161,3.907851,1.227618,4.434688,-11.122444,-0.609347,...,5.526824,-11.122444,-0.750304,3.937724,4.464742,5.024518,3.748742,-0.065698,-3.697836,4.102546
49,4.948433,2.242355,-1.188118,-11.122444,-0.431765,4.796046,0.653879,4.917976,-2.041450,0.266001,...,5.100068,-11.122444,-0.253467,2.709331,4.255977,4.634620,3.105601,1.724761,-3.353003,3.461577
87,4.506263,3.133793,0.326779,-11.122444,0.763496,3.853767,-0.333725,4.990898,-11.122444,0.207314,...,4.419336,-11.122444,-0.071146,3.470666,3.919040,4.314086,2.529508,0.332174,-1.170681,2.969810
121,5.025321,2.701623,0.985288,-11.122444,-0.818277,4.012470,3.221230,4.618955,-11.122444,-0.263575,...,4.682129,-11.122444,1.036041,3.327315,4.517733,4.496779,2.928541,0.001648,-4.159419,4.038905
105,4.684124,2.794659,1.101675,-11.122444,0.160606,5.128857,2.114092,4.454728,-11.122444,-0.217578,...,4.979373,-11.122444,-0.873107,3.607600,4.469733,4.322489,2.858515,0.317804,-1.721104,3.395859
23,4.523279,3.275668,0.508358,-11.122444,-0.837553,3.076831,3.051082,5.143041,-11.122444,-0.583468,...,4.222662,-11.122444,1.204022,3.364945,3.829530,4.064971,2.657011,-1.290408,-4.041191,2.886819


In [64]:
np.array(better_pathway_data)[:,0]

array([350, 1392, 1772, ..., 32016, 34450, 35175], dtype=object)